# Dataset pre processing

This notbook focuses on the bncc dataset pre processing aiming the training of a classification model for "etapa do conhecimento" prediction

In [1]:
import html

import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import (
    feature_extraction,
    linear_model,
    metrics,
    model_selection,
    preprocessing,
)

from src.data_preprocessing import cleaning

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pdsmoreau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)

## Selecting and filtering the features that will be used in the model

In [3]:
# Importing Data
df_bncc = pd.read_csv("/home/pdsmoreau/data-science/tera/bncc-classifier/data/raw/bncc_first_classifier.csv")

In [4]:
# Making a copy of the dataset and visualizing it
df_bncc_copy = df_bncc.copy()

df_bncc_copy.head()

,id,question,bulletType,name,slug,name.1,slug.1,name.2
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II


In [5]:
# Dropping unuseful columns
df_bncc_copy = df_bncc_copy[["id", "question", "name.1"]]

df_bncc_copy.head()

,id,question,name.1
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Espanhol
3,106855,<p><div> A respeito dos serviços públicos e da...,Direito Administrativo
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química


In [6]:
# Renaming dataset columns
df_bncc_copy.columns = ["id", "questions", "target"]

df_bncc_copy.head(3)

,id,questions,target
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Espanhol


In [7]:
# Observing the classes of the target to drop, aiming to comprehend only the topics of bncc
df_bncc_copy["target"].value_counts()

Matemática                                                                                                 12539
Língua Portuguesa                                                                                          11887
Geografia                                                                                                   9266
História                                                                                                    9072
Inglês                                                                                                      8103
Ciências                                                                                                    7099
Física                                                                                                      5537
Filosofia                                                                                                   3728
Arte                                                                                            

In [8]:
df_bncc_copy.shape

(100000, 3)

In [9]:
# Selecting observations which are the target are included on BNCC
df_bncc_copy_targets_bncc = df_bncc_copy.loc[
    df_bncc_copy["target"].isin(
        [
            "Língua Portuguesa",
            "Arte",
            "Educação Física",
            "Inglês",
            "Matemática",
            "Ciências",
            "Geografia",
            "História",
            "Ensino Religioso",
            "Biologia",
            "Física",
            "Química",
        ]
    )
]

Obs: in the setp of making the final solution, we must observe if the Topic of knowledge fits with the Setp of knowledge predicted.
For exemple: if the first code was EM, than, the topic must be Ciências Humanas
else EF , than the topic must be Geografia (or other in Cências Humanas)

In [10]:
# Observing if the filtering was correctly applied
df_bncc_copy_targets_bncc

,id,questions,target
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte
...,...,...,...
99994,2447693,<p>Leia a reportagem e responda a quest&atilde...,Língua Portuguesa
99995,4330761,"<p>Para responder à questão, leia o trecho do ...",Língua Portuguesa
99996,3249373,"<h1 style=""list-style:none;margin:0px 0px 20px...",Educação Física
99998,4150214,<p>Os machos e fêmeas de mosquitos do gênero <...,Ciências


In [11]:
# Observing if the filtering was correctly applied by visualizing the clases
df_bncc_copy_targets_bncc["target"].value_counts()

Matemática           12539
Língua Portuguesa    11887
Geografia             9266
História              9072
Inglês                8103
Ciências              7099
Física                5537
Arte                  3681
Química               3276
Biologia              3133
Educação Física       2386
Ensino Religioso       468
Name: target, dtype: int64

In [12]:
# How many observations do we have now?
df_bncc_copy_targets_bncc.shape

(76447, 3)

In [13]:
# Encoding the target with labels for the classifier
# this procedure can be done with LabelEncoder from scikit-learn
# will give us more control over the pipeline
lb_enc = preprocessing.LabelEncoder()

# train on the column we want encode
lb_enc.fit(df_bncc_copy_targets_bncc["target"])

# transform the same column, but here we'll have this transformation for test and after for train
df_bncc_copy_targets_bncc["target_enc"] = lb_enc.transform(
    df_bncc_copy_targets_bncc["target"]
)
df_bncc_copy_targets_bncc

/tmp/ipykernel_13532/1218335477.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc["target_enc"] = lb_enc.transform(


,id,questions,target,target_enc
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,0
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,10
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,11
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,7
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,0
...,...,...,...,...
99994,2447693,<p>Leia a reportagem e responda a quest&atilde...,Língua Portuguesa,9
99995,4330761,"<p>Para responder à questão, leia o trecho do ...",Língua Portuguesa,9
99996,3249373,"<h1 style=""list-style:none;margin:0px 0px 20px...",Educação Física,3
99998,4150214,<p>Os machos e fêmeas de mosquitos do gênero <...,Ciências,2


In [14]:
# you can access how the labels was transformed by looking at the classes
lb_enc.classes_

array(['Arte', 'Biologia', 'Ciências', 'Educação Física',
       'Ensino Religioso', 'Física', 'Geografia', 'História', 'Inglês',
       'Língua Portuguesa', 'Matemática', 'Química'], dtype=object)

## Cleaning the dataset

In [15]:
# chaining all cleaning steps
df_bncc_copy_targets_bncc["questions_clean"] = (
    df_bncc_copy_targets_bncc["questions"]
    .astype(str)
    .apply(html.unescape)
    .apply(lambda x: cleaning.remove_html(x))
    .apply(lambda x: x.lower())
    .apply(lambda x: cleaning.remove_punctuation_2(x))
    .apply(cleaning.remove_italic_quotes)
    .apply(cleaning.remove_open_quotes)
    .apply(cleaning.remove_end_quotes)
    .apply(cleaning.remove_italic_dquotes)
    .apply(cleaning.remove_open_dquotes)
    .apply(cleaning.remove_quote)
    .apply(lambda x: cleaning.remove_pt_stopwords(x))
    .apply(lambda x: cleaning.remove_en_stopwords(x))
)

/tmp/ipykernel_13532/3815083819.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc["questions_clean"] = (


In [16]:
df_bncc_copy_targets_bncc.head(10)

,id,questions,target,target_enc,questions_clean
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,0,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,10,achar número somado 2 igual inverso” equações ...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,11,chuva ácida fenômeno provocado poluição atmosf...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,7,pagina nr 54 livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,0,chiaroscuro
8,2610914,<p>Analise a igualdade a seguir</p><p><br></p>...,Matemática,10,analise igualdade seguiresta igualdade
12,1961666,"<p style=""text-align:justify"">Jo&atilde;o prec...",Matemática,10,joão precisa realizar movimentações financeira...
13,3612229,<p>Complete as lacunas.</p><p>________________...,Arte,0,complete lacunas apresentou desenho revolucion...
15,2423863,"<p>Analise o mapa a seguir.</p><div style=""tex...",Geografia,6,analise mapa seguira maior concentração espaci...
17,3121866,"<p>Observe a charge:</p> <p><a href=""https://3...",Geografia,6,observe charge retrata


In [17]:
# class to remove frq and rare, we can choose how many rare or frq words to remove
remove_frq_rare = cleaning.RemoveFrqRare(df=df_bncc_copy_targets_bncc)
remove_frq_rare.calc_frq_words()
remove_frq_rare.calc_rare_words()
bncc_cleaned_df = remove_frq_rare.remove_frq_and_rare()

In [18]:
bncc_cleaned_df.head()

,id,questions,target,target_enc,questions_clean
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,0,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,10,achar número somado 2 igual inverso” equações ...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,11,chuva ácida fenômeno provocado poluição atmosf...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,7,pagina nr 54 livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,0,chiaroscuro


### Exporting the dataset

In [19]:
bncc_cleaned_df = bncc_cleaned_df[["id", "questions_clean", "target_enc"]]

In [20]:
# Exporting
bncc_cleaned_df.to_csv(
    "/home/pdsmoreau/data-science/tera/bncc-classifier/data/curated/df_bncc_cleaned_model_two.csv"
)
